### Fuzzy_match_of_public
- Wang Jin & Zhongjie He

#### Problem describtion
The aim of this project is to pick the public companies(have listed on American stock exchanges) from the original 222,450 companies(some are repeated). The file **work1.csv** is the work file and the **work2** is the public companies(more than 6,000) in United States' share exchanges. Mainly we apply *fuzzy wuzzy* algorithm and picked the required ones. https://github.com/seatgeek/fuzzywuzzy (developped by SeatGeet)

**There are two main difficulties. **
- The first names in the file with unchecked companies are not very formal, say **'Apple company '** instead of **'Apple Inc'**. We designed the *fuzzy wuzzy* value by making experiments manually in advance. 
- The second is massive data leads to much running time (it is estimated more than **12 hours** for the first version programe). We reduced it to **less than five minutes** by devide the matching procedure into two steps after carefully observed the data structure.



In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import warnings
warnings.filterwarnings('ignore')
# fuzz.token_set_ratio("Advanced Semiconductor Engineering, Inc.", "Advance Pain Centers"    )

D:\Users\jinwa\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
#1.import company data we want search
co = pd.read_csv("work1.csv", sep=',')
# co.head()
co.describe()

,coname,money,year,comma,haha
count,222450,222450,222450,222447,222447
unique,41497,12710,39,1,537
top,Blue Cross/Blue Shield,",0,",2009,",",Y4000
freq,449,49595,9547,222447,27719


In [3]:
#import public companies data
share = pd.read_csv("work2.csv")
share.sort_values('sharename', inplace=True)
share.reset_index(inplace=True)
share.drop('index', axis=1, inplace=True)
share

,sharename,sharecode
0,"1-800 FLOWERS.COM, Inc.",FLWS
1,"1347 Property Insurance Holdings, Inc.",PIH
2,180 Degree Capital Corp.,TURN
3,1st Constitution Bancorp (NJ),FCCY
4,1st Source Corporation,SRCE
5,"21Vianet Group, Inc.",VNET
6,"22nd Century Group, Inc",XXII
7,"2U, Inc.",TWOU
8,3D Systems Corporation,DDD
9,3M Company,MMM


In [4]:
#2.make company name unique
co_name = co[['coname']]
co_name.drop_duplicates(inplace=True)
co_name.sort_values('coname', inplace=True)
co_name.reset_index(inplace=True)
co_name.drop(['index'], axis=1, inplace=True)
co_name.head()

,coname
0,1-800 Contacts
1,1-800-DoNotCall
2,1-800-Doctors
3,1-800-Flowers.com
4,1-800-Medivan


In [5]:
co_name.describe()

,coname
count,41497
unique,41497
top,National Milk Producers Federation
freq,1


In [6]:
#3. use fuzzy match algorithm 
import time
t0 = time.time()

pre=0
name2sharecode = {}

offset = 0
limit = len(co_name)

for j, co_row in co_name.iloc[offset: offset + limit].iterrows():
    cur = round((j - offset) / limit * 100)
    if cur > pre:
        pre = cur
        print(f'{cur}%...')   #give progress bar
    
    share_pre2 = share[share['sharename'].str.upper().str.startswith(co_row['coname'][0:2])]
    
    #match company names
    for i, share_row in share_pre2.iterrows():
        if fuzz.token_set_ratio(share_row['sharename'], co_row['coname']) > 75:
            name2sharecode[co_row['coname']] = share_row['sharecode']
            print(i, j)
            break
print(time.time() - t0)
import json
json.dump(name2sharecode, open('name2sharecode{}.json'.format(offset), 'w'))
name2sharecode

0 3
7 60
8 78
14 129
19 153
22 161
22 164
25 177
26 178
87 182
117 190
1.0%...
31 220
149 221
145 223
31 225
33 226
149 230
139 232
35 237
35 238
158 239
44 300
240 312
262 319
49 324
49 326
354 328
313 329
53 335
48 338
53 341
58 354
418 361
418 362
442 371
61 389
476 391
65 396
65 397
66 401
521 406
69 425
69 427
70 429
621 463
2.0%...
3.0%...
4.0%...
5.0%...
6.0%...
7.0%...
8.0%...
9.0%...
10.0%...
11.0%...
687 4399
762 4406
651 4417
651 4418
652 4427
656 4447
658 4452
668 4473
675 4500
675 4501
12.0%...
13.0%...
14.0%...
15.0%...
1029 6197
1029 6203
1032 6211
1029 6213
1029 6216
1173 6218
1204 6222
1046 6229
1046 6234
1267 6272
1053 6281
1058 6282
1060 6290
1060 6292
1303 6303
1062 6312
1069 6324
1072 6341
1072 6342
1073 6347
1075 6353
1077 6355
1075 6356
1075 6357
1078 6358
1479 6365
1615 6370
1082 6376
1083 6390
1083 6393
1090 6402
1090 6403
1091 6405
1657 6407
1094 6415
16.0%...
1106 6448
1109 6449
17.0%...
18.0%...
19.0%...
20.0%...
21.0%...
22.0%...
23.0%...
24.0%...
25.0%...


{'1-800-Flowers.com': 'FLWS',
 '2U Inc': 'TWOU',
 '3D Systems': 'DDD',
 '8X8 Inc': 'EGHT',
 'A10 Networks Government': 'ATEN',
 'AAI Corp': 'AIR',
 'AAR Corp': 'AIR',
 'ABIOMED Inc': 'ABMD',
 'ABM Industries': 'ABM',
 'ABS Global Inc': 'FCO',
 'ACA International': 'ATV',
 'ADA Technologies': 'AEY',
 'ADA-ES Inc': 'ADXSW',
 'ADCOR Industries': 'AEIS',
 'ADI Technologies': 'AEY',
 'ADOMANI Inc': 'ADOM',
 'ADS Inc': 'ADXSW',
 'ADSI Inc': 'ASIX',
 'AECOM Global': 'ACM',
 'AECOM Technology Corp': 'ACM',
 'AEGON NV': 'AEB',
 'AK Steel': 'AKS',
 'ALLETE Inc': 'ALE',
 'ALT Solutions': 'MDRX',
 'AM Networks Inc': 'AMCX',
 'AMC Networks': 'AMCX',
 'AMD Industries': 'ARII',
 'AMEC': 'AMFW',
 'AMG Healthcare Services': 'AMN',
 'AMI Entertainment': 'AMC',
 'AMN Healthcare Services': 'AMN',
 'ANSAR Inc': 'ANSS',
 'AON PLC': 'AON',
 'AON plc': 'AON',
 'API Technologies': 'AGTC',
 'ARCA Biopharma': 'ABIO',
 'ARCCA Inc': 'ARCH',
 'ARGUS INTERNATIONAL': 'ARRS',
 'ARMA International': 'ARRS',
 'ASA': 'AS

In [8]:
len(name2sharecode)

304

In [10]:
co['result'] = [name2sharecode.get(co_row['coname'], 'nopublic') for _, co_row in co.iterrows()]
co['result'].head()

#write back

0    nopublic
1    nopublic
2    nopublic
3    nopublic
4    nopublic
Name: result, dtype: object

In [11]:
co['result'].describe()

count       222450
unique         256
top       nopublic
freq        220249
Name: result, dtype: object

In [12]:
co[co['result'] != 'nopublic']

,coname,money,year,comma,haha,result
55,1-800-Flowers.com,",0,",2009,",",A8000,FLWS
56,1-800-Flowers.com,",30000,",2010,",",A8000,FLWS
195,2U Inc,",10000,",2015,",",Y4000,TWOU
196,2U Inc,",20000,",2016,",",Y4000,TWOU
247,3D Systems,",20000,",1998,",",Y4000,DDD
412,8X8 Inc,",20000,",2006,",",C4100,EGHT
413,8X8 Inc,",0,",2007,",",C4100,EGHT
414,8X8 Inc,",0,",2008,",",C4100,EGHT
462,A10 Networks Government,",35000,",2014,",",Y4000,ATEN
463,A10 Networks Government,",35000,",2015,",",Y4000,ATEN


In [13]:
co.to_csv("./jy_project.csv")
#write to csv